# 문제: 보배드림 사이트에서 중고차 100대 정보 추출  
  
  
차량명 인승 마력 연식 연료 주행 가격  
  
view_size=50  
page=2  
-> 100개 추출 & 데이터프레임 형태  

예)  
차량명: 현대 그랜져HG 300 LPG 모던  
인승: 5  
마력: 235  
연식: 15/11  
연료: LPG  
주행: 20만km  
가격: 880만원  

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm_notebook

###  한 페이지 & 차량 1대 정보  읽어오기

In [3]:
driver = webdriver.Chrome("c:/mychrome/chromedriver.exe")

In [4]:
url= "https://www.bobaedream.co.kr/cyber/CyberCar.php?gubun=K&page=3order=S11&view_size=50"
driver.get(url)

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# All 50 products on the page
print(len(soup.select("li.product-item")))
cars = soup.select("li.product-item")

50


In [7]:
# 1st car
car = cars[0]

In [8]:
# 차량명
car.select("p > a")[0].text

'기아 더 뉴 스포티지R 2.0 T-GDI 트렌디'

In [9]:
# 인승, 마력
spec_list = car.select("dl.data.clearfix > dd")
capacity = 0
hp = 0
for spec in spec_list:
    s = spec.text.strip()
    if '인승' in s:
        capacity = s.split('인승')[0]
    elif '마력' in s:
        hp = s.split('마력')[0]

In [10]:
# 연식
year = car.select("div.mode-cell.year > span")[0].text.strip().split('(')[0]

In [11]:
# 연료
fuel = car.select("div.mode-cell.fuel > span")[0].text.strip()

In [12]:
# 주행
km = car.select("div.mode-cell.km > span")[0].text.strip()

In [13]:
# 가격
price = car.select("div.mode-cell.price")[0].text.strip().replace('\n','')

In [14]:
price

'1,390만원'

### 100개 차량 정보 읽어오기

In [17]:
driver = webdriver.Chrome('c:/mychrome/chromedriver.exe')

In [18]:
# 보배드림 사이트 중고차 전체 검색에서 특정 페이지 & viewsize로 읽는 url 함수
def get_search_page_url(page, viewsize):
    return "https://www.bobaedream.co.kr/cyber/CyberCar.php?gubun=K&page={}&order=S11&view_size={}".format(page, viewsize)

In [19]:
# 페이지 검색 결과에 대한 차량 정보 추출하는 함수
def get_car_items(car_items):
    car_data=[]
    for car in car_items:
        # 차량명
        try:
            name=car.select("p > a")[0].text.strip()
        except:
            name=""
        # 인승, 마력
        try:
            spec_list = car.select("dl.data.clearfix > dd")
            capacity = 0
            hp = 0
            for spec in spec_list:
                s = spec.text.strip()
                if '인승' in s:
                    capacity = s.split('인승')[0]
                elif '마력' in s:
                    hp = s.split('마력')[0]
        except:
            capacity=""
            hp=""
        # 연식
        try:
            year = car.select("div.mode-cell.year > span")[0].text.strip().split('(')[0]
        except:
            year=""
        # 연료
        try:
            fuel = car.select("div.mode-cell.fuel > span")[0].text.strip()
        except:
            fuel=""
        # 주행
        try:
            km = car.select("div.mode-cell.km > span")[0].text.strip()
        except:
            km = ""
        # 가격
        try:
            price = car.select("div.mode-cell.price")[0].text.strip().replace('\n','').replace(',','')
        except:
            price=""
        car_data.append([name,capacity,hp,year,fuel,km,price])
    return car_data

In [20]:
page=2
viewsize=50
cars_data_total=[]

# 진행 상황 출력
for page in tqdm_notebook(range(1, page+1)):
    # 검색 페이지 이동
    url=get_search_page_url(page, viewsize)
    driver.get(url)
    time.sleep(3)
    
    # 현재 페이지 html 소스코드 가져오기
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    
    # 50개 중고차량 정보 추출
    cars_items = soup.select("li.product-item")
    car_item_list = get_car_items(cars_items)
    
    # 추출 정보 저장
    cars_data_total = cars_data_total + car_item_list

<ipython-input-20-47114bab082f>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1, page+1)):


In [21]:
len(cars_data_total)

100

In [22]:
cars_data_total

[['기아 더 뉴 모닝 밴 고급형', '2', '78', '16/09', '가솔린', '7만km', '398만원'],
 ['맥스 카니발 리무진', '9', '280', '16/12', '가솔린', '5만km', '3490만원'],
 ['현대 싼타페DM 2.0 VGT 2WD 프리미엄', 0, '184', '15/04', '디젤', '15만km', '1990만원'],
 ['현대 아반떼AD 1.6 T-GDi 스포츠 M/T', '5', '204', '17/06', '가솔린', '7만km', '1750만원'],
 ['기아 더 프레스티지 K7 3.0 LPi 프레스티지',
  '5',
  '235',
  '12/03',
  'LPG',
  '14만km',
  '1000만원'],
 ['현대 포터2 냉동탑차', 0, 0, '19/05', '디젤', '4만km', '2050만원'],
 ['쉐보레 스파크 LT', '5', '70', '13/10', '가솔린', '3만km', '290만원'],
 ['현대 제네시스 쿠페 380 GT R', '4', '303', '10/04', '가솔린', '11만km', '690만원'],
 ['현대 그랜져IG 2.2 e-VGT 프리미엄 스페셜', '5', '202', '17/11', '디젤', '3만km', '3150만원'],
 ['기아 K3 1.6 GDI 럭셔리', '5', '140', '15/08', '가솔린', '3만km', '920만원'],
 ['기아 레이 1.0 가솔린 디럭스', '5', '78', '16/09', '가솔린', '12만km', '420만원'],
 ['기아 K5 2.0 LPi 럭셔리', '5', '144', '11/11', 'LPG', '19만km', '300만원'],
 ['기아 더 뉴 K5 2.0 트렌디', '5', '172', '14/03', '가솔린', '11만km', '1090만원'],
 ['기아 K5 2.0 LPi 프레스티지', '5', '144', '11/04', 'LPG', '16만km', '750만원'],
 ['

### 추출한 데이터를 데이터프레임에 저장

In [23]:
data = pd.DataFrame(cars_data_total, columns=['차량명', '인승', '마력', '연료', '연식', '주행', '가격'])

In [24]:
data

,차량명,인승,마력,연료,연식,주행,가격
0,기아 더 뉴 모닝 밴 고급형,2,78,16/09,가솔린,7만km,398만원
1,맥스 카니발 리무진,9,280,16/12,가솔린,5만km,3490만원
2,현대 싼타페DM 2.0 VGT 2WD 프리미엄,0,184,15/04,디젤,15만km,1990만원
3,현대 아반떼AD 1.6 T-GDi 스포츠 M/T,5,204,17/06,가솔린,7만km,1750만원
4,기아 더 프레스티지 K7 3.0 LPi 프레스티지,5,235,12/03,LPG,14만km,1000만원
...,...,...,...,...,...,...,...
95,쉐보레 더 뉴 스파크 1.0 LT,5,75,18/11,가솔린,1만km,850만원
96,르노삼성 SM3 네오 SE 플러스,5,117,14/06,가솔린,12만km,530만원
97,현대 그레이스 LPG 15인승 그랜드 살롱,15,0,02/03,LPG,13만km,계약
98,현대 아반떼MD M16 GDi 스마트,5,140,13/04,가솔린,4만km,700만원
